In [4]:
import numpy as np
import os
from os.path import join, exists, split
import glob
import pandas as pd

In [2]:
homedir = split(os.getcwd())[0]
datadir = join(homedir,'miniblock','derivatives')
presdir = join(homedir, 'Behavior', 'CondRichData')
outdir  = join(homedir, 'Behavior', 'designmats')

In [3]:
subjects = ["19"]
max_runs = 9

In [4]:
for sub in subjects: 
    for run in range(1,max_runs+1):
        pattern = presdir + f'/P0{sub}_ConditionRich_Run{run}*.csv'
        matches = glob.glob(pattern)
        match = matches[0]
        df = pd.read_csv(match)
        df = df.drop([0, 1]).reset_index(drop=True)
        # Check condition
        if df["imFile"][5] == df ["imFile"][7]:
            runtype = "miniblock"
            duration = 4
        elif (df["eventEndTime"][3] - df["eventStartTime"][3]) == 3.75 :
            runtype = "sus"
            duration = 4
        else: 
            runtype = "er"
            duration = 1
        file_names = df["imFile"].unique()
        file_names = file_names[file_names != 'Stimuli/Blank.png']
        file_names.sort()
        # print(file_names)
        empty_array = np.zeros((388,40))

        df['two_before'] = df['imFile'].shift(2)

        for name in range(empty_array.shape[1]):
            for row in range(len(df["imFile"])):
                if (file_names[name] == df["imFile"][row]) & (df["imFile"][row] != df["two_before"][row]): 
                    empty_array[int(df["eventStartTime"][row]), name] = 1

        path = outdir + f'/P0{sub}_ConditionRich_Run_0{run}_{runtype}.csv'
        np.savetxt(path, empty_array, delimiter=",", fmt="%d")

## The same for localizer data

In [12]:
homedir = "/Users/danieljanini/Documents/Thesis/"
datadir = join(homedir,'miniblock','derivatives')
presdir = join(homedir, 'Behavior', 'LocData')
outdir  = join(homedir, 'Behavior', 'designmats', 'localizer')

In [44]:
subjects = ["01","02", "03","04", "05","06", "07", "08", "10", "11", "12", "13", "14", "15", "17", "18", "19"]
max_runs = 3

In [85]:
for sub in subjects: 
    for run in range(1,max_runs+1):
        pattern = presdir + f'/P0{sub}_CategoryLocalizer_Run{run}*.csv'
        matches = glob.glob(pattern)
        if matches == []:
            print(f"Run {run} not found for subject {sub}. Skipping.")
            continue
        match = matches[0]
        df = pd.read_csv(match)
        df = df.drop([0, 1]).reset_index(drop=True)
        df["Faces"]= df["imFile"].str.contains("Faces")
        df["Objects"]= df["imFile"].str.contains("Objects")
        df["Scenes"] = df["imFile"].str.contains("Scenes")
        df["Bodies"]= df["imFile"].str.contains("Bodies")

        categories = ["Faces", "Objects", "Scenes", "Bodies"]
        
        # print(file_names)
        empty_array = np.zeros((int(max(df["eventEndTime"])),4))

        df['two_before'] = df['imFile'].shift(2)

        for name in range(4):
            for row in range(len(df["imFile"])):
                if df[categories[name]][row] and df[categories[name]][row-1] == False: 
                    empty_array[int(df["eventStartTime"][row]), name] = 1

        rows, cols = empty_array.shape

        new_array = np.zeros(empty_array.shape)

        for col in range(cols):
            for row in range(rows):
                if empty_array[row][col] == 1:
                    if empty_array[row-1][col] == 1:
                        new_array[row][col] = 0
                    else: 
                        new_array[row][col] = 1
                else: 
                    new_array[row][col] = 0

        path = outdir + f'/P0{sub}_CategoryLocalizer_Run_0{run}.csv'
        np.savetxt(path, new_array, delimiter=",", fmt="%d")

Run 3 not found for subject 01. Skipping.
Run 3 not found for subject 02. Skipping.
Run 3 not found for subject 05. Skipping.
Run 3 not found for subject 11. Skipping.
Run 3 not found for subject 12. Skipping.


In [48]:
df[categories[name]][row-1] == False

True

In [83]:
rows, cols = empty_array.shape

new_array = np.zeros(empty_array.shape)

for col in range(cols):
    for row in range(rows):
        if empty_array[row][col] == 1:
            if empty_array[row-1][col] == 1:
                new_array[row][col] = 0
            else: 
                new_array[row][col] = 1
        else: 
            new_array[row][col] = 0
            

In [84]:
new_array.sum()

16.0